In [5]:
import numpy as np
import games as games
import game_utils as utils
import game_utils_external_samp as external_sampling
import game_tree as game_tree 
from importlib import reload
reload(utils)
reload(games)
reload(game_tree)
reload(external_sampling)

import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import List

In [2]:
# For test purposes
# game_tfdp = game_tree.GameTFDP(game_class=games.RockPaperSuperScissors)
# game_tfdp.build_tfdp_from_file(filename='rps_player0.txt', player=0)
# game_tfdp.build_tfdp_from_file(filename='rps_player1.txt', player=1)

game_tfdp = game_tree.GameTFDP(game_class=games.PhantomTicTacToe)
game_tfdp.build_tfdp_from_file(filename='player0-infoset.txt', player=0)
game_tfdp.build_tfdp_from_file(filename='player1-infoset.txt', player=1)

Initializing Infosets...


100%|██████████| 14482810/14482810 [01:36<00:00, 149540.94it/s]


Making TFDP...


100%|██████████| 14482810/14482810 [00:16<00:00, 883411.71it/s]


Initializing Infosets...


100%|██████████| 8827459/8827459 [00:57<00:00, 154294.69it/s]


Making TFDP...


100%|██████████| 8827459/8827459 [00:09<00:00, 911949.57it/s]


In [11]:
import zipfile
import datetime
from glob import glob

def save_zip_file(numpy_tensor_filename, player: int):
    zipfilename = f'pttt_pl{player}.zip'
    with zipfile.ZipFile(zipfilename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(numpy_tensor_filename)
    print(f"Zip file '{zipfilename}' created successfully.")
    

def save_policy(policy: game_tree.SparseSeqVec, player: int, game_tfdp: game_tree.GameTFDP, 
                infoset_filename: str, policy_name: str = ""):
    lines = open(infoset_filename, "r").readlines()
    array = np.zeros((len(lines), game_tfdp.game_class.NUM_ACTIONS))
    for i, line in enumerate(lines):
        infoset_label = line.strip()
        infoset = game_tfdp.infoset_map[player][infoset_label]
        dist = utils.normalize(policy[infoset])
        for action in infoset.action_to_idx:
            array[i][action] = dist[infoset.action_to_idx[action]]
            
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    if len(policy_name) > 0:
        filename = "./policies/" + policy_name + f"{player}.npy"
    else:
        filename = "./policies/" + f"pttt_pl{player}_{timestamp}.npy"
    np.save(filename, array)
    print(f"Saved '{filename}' successfully.")
    save_zip_file(filename, player)

def load_policy(numpy_tensor_filename: str, infoset_filename: str, game_tfdp: game_tree.GameTFDP, player: int):
    policy = game_tree.SparseSeqVec(game_tfdp, player)
    array = np.load(numpy_tensor_filename)
    lines = open(infoset_filename, "r").readlines()
    for i, line in enumerate(tqdm(lines)):
        infoset_label = line.strip()
        infoset = game_tfdp.infoset_map[player][infoset_label]
        vec = np.zeros(len(infoset.idx_to_action))
        for j in infoset.idx_to_action:
            vec[j] = array[i][infoset.idx_to_action[j]]
        policy[infoset] = vec
    return policy

def get_latest_policy_filenames(num_players: int):
    policy_files = ["" for _ in range(num_players)]
    for player in range(num_players):
        policy_files[player] = sorted(glob(f"./policies/pttt_pl{player}_*.npy"))[-1]
    return policy_files

In [ ]:
# learn Nash Equilibria (Outcome Sampling) (call this)

def outcome_sampling_learner(game_tfdp, load_policy_files: List[str] = []):
    if load_policy_files:
        average_policies = [None for _ in range(game_tfdp.num_players)]
        for player in range(game_tfdp.num_players):
            average_policies[player] = load_policy(load_policy_files[player], f"player{player}-infoset.txt", game_tfdp, player)
        current_policies = [average_policies[player].copy() for player in range(game_tfdp.num_players)]
    else:
        current_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize) for player in range(game_tfdp.num_players)]
        average_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize) for player in range(game_tfdp.num_players)]

    cum_regret = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
    cum_reaches = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
    print("Expectation")
    ev_list = [utils.compute_expected_value(game_tfdp, 0, average_policies)]
    print("Nash Gap")
    gap_list = [utils.nash_conv(game_tfdp, average_policies)[0]]
    T = 10000 # change this to your liking
    pbar = tqdm(range(T))
    for episode in pbar:
        tfdp_subtrees = [{}, {}]
        for player in range(game_tfdp.num_players):
            cf_value, tfdp_subtrees[player] = utils.compute_counterfactual_value(game_tfdp, player, current_policies)
            cf_regret = cf_value - (cf_value * current_policies[player]).reduce(np.sum).sparseseqvec()
            current_policies[player] = utils.CFR(game_tfdp, player, current_policies, cf_regret, cum_regret[player], tfdp_subtrees[player])

        for player in range(game_tfdp.num_players):
            cum_reaches[player] = utils.get_reach_probability(game_tfdp, player, current_policies[player], tfdp_subtrees[player]) / (episode + 1.) \
                + cum_reaches[player] * episode / (episode + 1.)
            average_policies[player] = cum_reaches[player].map(utils.normalize)

        iir = 0.1
        ev = iir * utils.compute_expected_value(game_tfdp, 0, average_policies) + ev_list[-1] * (1-iir)
        nash_gap = iir * utils.nash_conv(game_tfdp, average_policies)[0] + gap_list[-1] * (1-iir) # exponential averaging
        ev_list.append(ev)
        gap_list.append(nash_gap)
        
        pbar.set_description(f"Expected utility {ev:.5f}, Nash gap: {nash_gap:.5f}")
    pbar.close()

    for player in range(game_tfdp.num_players):
        save_policy(average_policies[player], player, game_tfdp, f"player{player}-infoset.txt")
        
    
    return average_policies, ev_list, gap_list

In [ ]:
average_policies, ev_list, gap_list = outcome_sampling_learner(game_tfdp)

In [12]:
# tree pruning (Works :D)
load_policy_files = get_latest_policy_filenames(game_tfdp.num_players)
average_policies = [None for _ in range(game_tfdp.num_players)]
for player in range(game_tfdp.num_players):
    print(f"Loading policy for player {player} from {load_policy_files[player]}")
    average_policies[player] = load_policy(load_policy_files[player], f"player{player}-infoset.txt", game_tfdp, player)
    print("Done.")

utils.prune_tfdp(game_tfdp, average_policies)

Loading policy for player 0 from ./policies\pttt_pl0_20241128_045719.npy


100%|██████████| 14482810/14482810 [01:39<00:00, 145733.72it/s]


Done.
Loading policy for player 1 from ./policies\pttt_pl1_20241128_045919.npy


100%|██████████| 8827459/8827459 [01:03<00:00, 139199.57it/s]


Done.
Initial Num nodes: [14482810, 8827459], After pruning: [1570499, 4099189]


In [14]:
# learn Nash Equilibria with external sampling (call This)
# better to run after pruning the game_TFDP
# this method is expected to give better performance

def external_sampling_learner(game_tfdp: game_tree.GameTFDP, load_policy_files: List[str] = []):
    if load_policy_files:
        average_policies = [None for _ in range(game_tfdp.num_players)]
        for player in range(game_tfdp.num_players):
            average_policies[player] = load_policy(load_policy_files[player], f"player{player}-infoset.txt", game_tfdp, player)
        current_policies = [average_policies[player].copy() for player in range(game_tfdp.num_players)]
    else:
        current_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize) for player in range(game_tfdp.num_players)]
        average_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize) for player in range(game_tfdp.num_players)]

    cum_regret = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
    cum_reaches = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
    print("Expectation")
    ev_list = [external_sampling.compute_expected_value(game_tfdp, 0, average_policies)]
    print("Nash Gap")
    gap_list = [external_sampling.nash_conv(game_tfdp, average_policies)[0]]
    T = 50 # change this to your liking, start with small numbers
    pbar = tqdm(range(T))
    for episode in pbar:
        for player in range(game_tfdp.num_players):
            cf_value = external_sampling.compute_counterfactual_value(game_tfdp, player, current_policies)
            cf_regret = cf_value - (cf_value * current_policies[player]).reduce(np.sum).sparseseqvec()
            current_policies[player] = external_sampling.CFR(game_tfdp, player, current_policies, cf_regret, cum_regret[player])

        for player in range(game_tfdp.num_players):
            cum_reaches[player] = external_sampling.get_reach_probability(game_tfdp, player, current_policies[player]) / (episode + 1.) \
                + cum_reaches[player] * episode / (episode + 1.)
            average_policies[player] = cum_reaches[player].map(utils.normalize)

        iir = 0.1
        ev = iir * external_sampling.compute_expected_value(game_tfdp, 0, average_policies) + ev_list[-1] * (1-iir)
        nash_gap = iir * external_sampling.nash_conv(game_tfdp, average_policies)[0] + gap_list[-1] * (1-iir) # exponential averaging
        ev_list.append(ev)
        gap_list.append(nash_gap)
        
        pbar.set_description(f"Expected utility {ev:.5f}, Nash gap: {nash_gap:.5f}")
    pbar.close()  
    
    for player in range(game_tfdp.num_players):
        save_policy(average_policies[player], player, game_tfdp, f"player{player}-infoset.txt")

    return average_policies, ev_list, gap_list

In [ ]:
average_policies, ev_list, gap_list = external_sampling_learner(game_tfdp, get_latest_policy_filenames(game_tfdp.num_players))


In [ ]:
# plot
fig, ax = plt.subplots()
ax.set_xlabel('iteration')
ax.set_ylabel('utility')
ax.plot(ev_list)

fig, ax = plt.subplots()
ax.set_xlabel('iteration')
ax.set_ylabel('Nash gap')
ax.plot(gap_list)


In [ ]:
# Test - Learning against Uniform policy
print("Loading Current Policy")
current_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize) for player in range(game_tfdp.num_players)]
cum_regret = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
cum_reaches = [game_tree.SparseSeqVec(game_tfdp, player) for player in range(game_tfdp.num_players)]
print("Loading Average Policy")
average_policies = [game_tree.SparseSeqVec(game_tfdp, player).map(utils.normalize)  for player in range(game_tfdp.num_players)]
print("Computing Expected value")
ev_list = [utils.compute_expected_value(game_tfdp, 0, average_policies)]
print(ev_list)

T = 200
pbar = tqdm(range(T))
for episode in pbar:
    player = 0
    cf_value, tfdp_subtree = utils.compute_counterfactual_value(game_tfdp, player, current_policies)
    cf_regret = cf_value - (cf_value * current_policies[player]).reduce(np.sum).sparseseqvec()
    current_policies[player] = utils.CFR(game_tfdp, player, current_policies, cf_regret, cum_regret[player], tfdp_subtree)

    cum_reaches[player] = utils.get_reach_probability(game_tfdp, player, current_policies[player], tfdp_subtree) / (episode + 1.) \
        + cum_reaches[player] * episode / (episode + 1.)
    average_policies[player] = cum_reaches[player].map(utils.normalize)

    ev = utils.compute_expected_value(game_tfdp, 0, average_policies)
    ev_list.append(ev)

    pbar.set_description(f"Expected utility {ev:.5f}")
pbar.close()